# Introduction

The notebook is intended to perform data cleaning process over the dataset **Customer Personality Analysis**.


[Dataset](https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis)

In [3]:
# Import Standard Modules
import pandas as pd

import plotly.express as ex

# Set Pandas Options
pd.set_option('display.max_columns', 500)

# Read Data

In [4]:
# Read data from csv
data = pd.read_csv('../data/marketing_campaign.csv', sep='\t', encoding='latin1')

# Cleaning Outliers

Possible approaches:
1. Drop outliers - This technique can drastically reduce the amount of data.
2. Cap outliers - This technique is useful when we can assume that all outliers epress the same behaviors or pattern and thus the model wouldn't learn anything new from them
3. Fill using mean

## Interquartile Range (IQR)

In [5]:
# Compute Q1 and Q3
q1 = data.quantile(0.25)
q3 = data.quantile(0.75)

In [6]:
# Compute the IQR
iqr = q3 - q1

In [10]:
# Filter out outliers based on IQR
outliers = data < (q1 - 1.5*iqr) | data > (q3 + 1.5*iqr)

TypeError: unsupported operand type(s) for |: 'float' and 'int'

In [9]:
len(outliers)

2240

In [ ]:
len(data)